In [1]:
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import importlib
import warnings

warnings.filterwarnings('ignore')
from tensorflow.keras.models import load_model
import importlib
import Model
importlib.reload(Model)

import utils
import config

def train_model(path, classes, train_ratio):
    feature_set, labels = utils.get_dataset(path, classes, show_labels=False)
    X_train, y_train, X_test, y_test = utils.split_data(feature_set, labels, ratio=train_ratio)
    CNN_decoder = Model.Original_model_V1(X_train.shape[1:])

    history, CNN_decoder = Model.Train_model(CNN_decoder, X_train, y_train, X_test, y_test,
                      set_epoch=50, set_batch_size=256, Model_name='V0', set_verbose=False, save_model_set=True)
    acc = CNN_decoder.evaluate(X_test, y_test, verbose=0)[1]
    print(f"Accuracy of test dataset using model V0 (trained with only 5/27 data): {acc*100:.4f}%")

    return CNN_decoder, float(np.max(history.history['val_accuracy']))*100


def testing_new_session(path, classes, CNN_decoder, train_ratio):
    feature_set, labels = utils.get_dataset(path, classes, show_labels=False)
    _, _, X_test, y_test = utils.split_data(feature_set, labels, ratio=train_ratio)
    acc = CNN_decoder.evaluate(X_test, y_test, verbose=0)[1]
    print(f"Accuracy of unseen dataset using model V0 (trained with only 5/27 data): {acc*100:.4f}%")

    return float(np.round(acc*100, 2))


default_path = config.default_path
classes = config.classes
LST = []

'''version = 'V0'
CNN_decoder = load_model(f'best_model_{version}.keras')'''


for idx, session_info in enumerate(config.dataset):
    print(f"===========================================\nDataset {idx+1}/{len(config.dataset)} - Session {session_info}\n===========================================\n")
    path = default_path + f'{session_info}/raw/'

    if idx == 0:
        CNN_decoder, acc = train_model(path, classes, train_ratio=0.8)
    else:
        acc = testing_new_session(path, classes, CNN_decoder, train_ratio=0)  #This is only for testing

    LST.append(acc)


df = pd.DataFrame({
    'Info': config.Info,
    'Info_Set': config.dataset,
    'Accuracy': LST
})

# Save to CSV
df.to_csv('C:/Users/hml76/PycharmProjects/Tremor_project_local/Mindforce/Baseline_results.csv', index=False)
df

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings('ignore')

import Model
import utils
import config

# Reload in case of edits
importlib.reload(Model)


class TremorModelTrainer:
    def __init__(self, config):
        self.default_path = config.default_path
        self.classes = config.classes
        self.dataset_info = config.dataset
        self.info_labels = config.Info
        self.results = []
        self.model = None


    def train(self, path, train_ratio=0.8):
        feature_set, labels = utils.get_dataset(path, self.classes, show_labels=False)
        X_train, y_train, X_test, y_test = utils.split_data(feature_set, labels, ratio=train_ratio)

        self.model = Model.Original_model_V1(X_train.shape[1:])

        history, self.model = Model.Train_model(
            self.model, X_train, y_train, X_test, y_test,
            set_epoch=50, set_batch_size=256, Model_name='V0',
            set_verbose=False, save_model_set=True
        )

        acc = self.model.evaluate(X_test, y_test, verbose=0)[1]
        print(f"Accuracy of test dataset using model V0: {acc * 100:.4f}%")
        return float(np.max(history.history['val_accuracy']) * 100)


    def test(self, path):
        feature_set, labels = utils.get_dataset(path, self.classes, show_labels=False)
        _, _, X_test, y_test = utils.split_data(feature_set, labels, ratio=0)
        acc = self.model.evaluate(X_test, y_test, verbose=0)[1]
        print(f"Accuracy on unseen dataset: {acc * 100:.4f}%")
        return float(np.round(acc * 100, 2))


    def run_all_sessions(self):
        for idx, session_info in enumerate(self.dataset_info):
            print(f"\n{'='*40}\nDataset {idx + 1}/{len(self.dataset_info)} - Session {session_info}\n{'='*40}")
            path = os.path.join(self.default_path, f'{session_info}/raw/')

            if idx == 0:
                acc = self.train(path)
            else:
                acc = self.test(path)

            self.results.append(acc)


    def save_results(self, filepath):
        df = pd.DataFrame({
            'Info': self.info_labels,
            'Info_Set': self.dataset_info,
            'Accuracy': self.results
        })
        df.to_csv(filepath, index=False)
        print(f"Results saved to {filepath}")
        return df


In [3]:
if __name__ == "__main__":
    trainer = TremorModelTrainer(config)
    trainer.run_all_sessions()
    df = trainer.save_results('../Results/Baseline_results.csv')



Dataset 1/22 - Session Exp_2025-05-27/E8331D05289A/
Start Training (total epochs: 50)...
Finish Training! (Model is saved)
Maximum training accuracy : 90.7%
Maximum validation accuracy : 94.55%
Accuracy of test dataset using model V0: 94.2609%

Dataset 2/22 - Session Exp_2025-06-18/E9AD0E7DCC2B/
Accuracy on unseen dataset: 83.0530%

Dataset 3/22 - Session Exp_2025-06-20-v1/E9AD0E7DCC2B/
Accuracy on unseen dataset: 80.3447%

Dataset 4/22 - Session Exp_2025-06-20-v2/E9AD0E7DCC2B/
Accuracy on unseen dataset: 87.0545%

Dataset 5/22 - Session Exp_2025-06-20-v3/E9AD0E7DCC2B/
Accuracy on unseen dataset: 84.8836%

Dataset 6/22 - Session Exp_2025-06-20-v4/E9AD0E7DCC2B/
Accuracy on unseen dataset: 73.5952%

Dataset 7/22 - Session Exp_2025-06-20-v5/E9AD0E7DCC2B/
Accuracy on unseen dataset: 71.4901%

Dataset 8/22 - Session Exp_2025-06-20-v6/E9AD0E7DCC2B/
Accuracy on unseen dataset: 74.8581%

Dataset 9/22 - Session Exp_2025-06-20-v7/E9AD0E7DCC2B/
Accuracy on unseen dataset: 67.6662%

Dataset 10/22